In [1]:
'''
The code is used to train BC imitator, or pretrained GAIL imitator
'''

import argparse
import tempfile
import os.path as osp
import gym
import logging
from tqdm import tqdm

import tensorflow as tf

from baselines.gail import mlp_policy
from baselines import bench
from baselines import logger
from baselines.common import set_global_seeds, tf_util as U
from baselines.common.misc_util import boolean_flag

#from baselines.gail.run_mujoco import runner
from baselines.gail.dataset.mujoco_dset import Mujoco_Dset 

##from baselines.common.mpi_adam import MpiAdam

In [2]:
def generate_expert_data_with_rewards(scenario_file, num_interactions=1000, file_name="expert_data", act_ndim=19):
    #from gfrl.base.run_my_ppo2 import create_single_scenic_environment
    import numpy as np
    expert_observations = []
    expert_actions = []
    expert_rewards = []

    gf_env_settings = {
        "stacked": True,
        "rewards": 'scoring',
        "representation": 'extracted',
        "players": [f"agent:left_players=1"],
        "action_set": "default",#"default" "v2"
    }

    from scenic.simulators.gfootball.rl_interface import GFScenicEnv
    from scenic.simulators.gfootball.utilities.scenic_helper import buildScenario
    scenario = buildScenario(scenario_file)
    env = GFScenicEnv(initial_scenario=scenario, gf_env_settings=gf_env_settings, use_scenic_behavior_in_step=True, constraints_checking=True)
    
    obs = env.reset()
    tr = 0
    for i in tqdm(range(num_interactions)):
        expert_observations.append(obs)

        obs, reward, done, info = env.step(env.action_space.sample())
        tr+=reward
        # print(info)
        action = info["action_taken"]
        expert_actions.append(action)

        if done:
            obs = env.reset()
            expert_rewards.append(tr)
            tr = 0

    expert_observations = np.array(expert_observations)
    #expert_observations = np.moveaxis(expert_observations, [3], [1])
    acts = np.array(expert_actions)

    acts_oh = np.zeros((acts.shape[0], act_ndim))
    acts_oh[np.arange(acts.shape[0]), acts] = 1

    expert_rewards = np.array(expert_rewards)
    print("Expert observation shape: ", expert_observations.shape)
    print("Expert actions shape: ", acts_oh.shape)
    print("Num Episode: ", expert_rewards.shape[0])
    print("Mean Reward: ", expert_rewards.mean())

    #np.savez(expert_file_name, obs=mb_obs, acs=mb_act_oh, num_epi = num_episodes, mean_reward = np.sum(mb_rewards)/num_episodes)
    np.savez_compressed(
        file_name,
        acs=acts_oh,
        obs=expert_observations,
        num_epi = expert_rewards.shape[0],
        mean_reward = expert_rewards.mean(),
        rewards = expert_rewards
    )
    return expert_observations, acts_oh, expert_rewards



In [50]:
def generate_expert_successful_data(scenario_file, num_interactions=1000, file_name="expert_data", act_ndim=19):
    #from gfrl.base.run_my_ppo2 import create_single_scenic_environment
    import numpy as np
    expert_observations = []
    expert_actions = []
    expert_rewards = []

    gf_env_settings = {
        "stacked": True,
        "rewards": 'scoring',
        "representation": 'extracted',
        "players": [f"agent:left_players=1"],
        "action_set": "default",#"default" "v2"
    }

    from scenic.simulators.gfootball.rl_interface import GFScenicEnv
    from scenic.simulators.gfootball.utilities.scenic_helper import buildScenario
    scenario = buildScenario(scenario_file)
    env = GFScenicEnv(initial_scenario=scenario, gf_env_settings=gf_env_settings, use_scenic_behavior_in_step=True, constraints_checking=True)
    
    
    tr = 0
    
    obs_buf, acts_buf, rew_buf  = [], [], []

    
    with tqdm(total=num_interactions) as pbar:
        
        while(len(expert_observations)<num_interactions):
            
            
            if len(obs_buf)==0:
                obs = env.reset()
            
            obs_buf.append(obs)
            obs, reward, done, info = env.step(env.action_space.sample())
            
            #rew_buf.append(reward)

            tr+=reward
            # print(info)
            action = info["action_taken"]
            acts_buf.append(action)

            if done:
                
                #print(f"New Epi: {len(obs_buf)} R: {tr}")
                
                if tr>0: 
                    expert_observations.extend(obs_buf)
                    expert_actions.extend(acts_buf)
                    expert_rewards.append(tr)
                    
                    if len(expert_observations)> num_interactions:
                        pbar.update(num_interactions)
                    else:
                        pbar.update(len(obs_buf)) 
                        
                    #print("Added new Api. Current Size: ", len(expert_observations))


                obs_buf, acts_buf, rew_buf  = [], [], []
                obs = env.reset()
                tr = 0

            
    print("Collection Done")        
        

    expert_observations = np.array(expert_observations)
    #expert_observations = np.moveaxis(expert_observations, [3], [1])
    acts = np.array(expert_actions)

    acts_oh = np.zeros((acts.shape[0], act_ndim))
    acts_oh[np.arange(acts.shape[0]), acts] = 1

    expert_rewards = np.array(expert_rewards)
    
    print("Expert observation shape: ", expert_observations.shape)
    print("Expert actions shape: ", acts_oh.shape)
    print("Num Episode: ", expert_rewards.shape[0])
    print("Mean Reward: ", expert_rewards.mean())

    #np.savez(expert_file_name, obs=mb_obs, acs=mb_act_oh, num_epi = num_episodes, mean_reward = np.sum(mb_rewards)/num_episodes)
    np.savez_compressed(
        file_name,
        acs=acts_oh,
        obs=expert_observations,
        num_epi = expert_rewards.shape[0],
        mean_reward = expert_rewards.mean(),
        rewards = expert_rewards
    )
    return expert_observations, acts_oh, expert_rewards



In [51]:
from gfrl.base.run_my_ppo2 import create_single_scenic_environment

scenario = "../_scenarios/academy/wb/pass_n_shoot_wb.scenic"

num_interactions=10000
expert_file_name = f"../_data/pns_success_{num_interactions}.npz"

obs, act, rew = generate_expert_successful_data(scenario_file=scenario, num_interactions=num_interactions, file_name=expert_file_name)



  0%|          | 0/10000 [00:00<?, ?it/s]

Environment will ignore actions passed to step() and take action provided by Scenic


  0%|          | 32/10000 [00:02<12:42, 13.08it/s]

New Epi: 32 R: 1.0
Added new Api. Current Size:  32


  1%|          | 67/10000 [00:06<15:34, 10.63it/s]

...

New Epi: 35 R: 1.0
Added new Api. Current Size:  9974
New Epi: 32 R: 0.0
New Epi: 44 R: 0.0


19974it [43:36, 191.12it/s]                         

New Epi: 33 R: 1.0
Added new Api. Current Size:  10007


19974it [43:38,  7.63it/s] 


Collection Done
Expert observation shape:  (10007, 72, 96, 16)
Expert actions shape:  (10007, 19)
Num Episode:  297
Mean Reward:  1.0


In [52]:
from gfrl.common.mybase.cloning.dataset import GFDset
expert_file_name = f"../_data/pns_success_{num_interactions}.npz"
dataset = GFDset(expert_file_name)
print(dataset.num_epi, dataset.mean_reward, dataset.size, dataset.acts.shape)

297 1.0 10007 (10007, 19)


In [4]:
from gfrl.base.run_my_ppo2 import create_single_scenic_environment

scenario = "../_scenarios/academy/wb/pass_n_shoot_wb.scenic"

num_interactions=1000
expert_file_name = f"../_data/pns_{num_interactions}.npz"

obs, act, rew = generate_expert_data_with_rewards(scenario_file=scenario, num_interactions=num_interactions, file_name=expert_file_name)

Environment will ignore actions passed to step() and take action provided by Scenic


100%|██████████| 1000/1000 [00:08<00:00, 111.66it/s]


Expert observation shape:  (1000, 72, 96, 16)
Expert actions shape:  (1000, 19)
Num Episode:  27
Mean Reward:  0.5925925925925926


In [5]:
import numpy as np
td = np.load(expert_file_name)

In [6]:
list(td.keys())

['acs', 'obs', 'num_epi', 'mean_reward', 'rewards']

In [8]:
scenario = "../_scenarios/academy/wb/pass_n_shoot_wb.scenic"

num_interactions=5000
expert_file_name = f"../_data/pns_{num_interactions}.npz"

obs, act, rew = generate_expert_data_with_rewards(scenario_file=scenario, num_interactions=num_interactions, file_name=expert_file_name)

from gfrl.common.mybase.cloning.dataset import GFDset
dataset = GFDset(expert_file_name)
print(dataset.num_epi, dataset.mean_reward, dataset.size, dataset.acts.shape)


  0%|          | 0/5000 [00:00<?, ?it/s]

Environment will ignore actions passed to step() and take action provided by Scenic


100%|██████████| 5000/5000 [00:59<00:00, 83.51it/s] 


Expert observation shape:  (5000, 72, 96, 16)
Expert actions shape:  (5000, 19)
Num Episode:  133
Mean Reward:  0.6240601503759399
133 0.6240601503759399 5000 (5000, 19)


In [9]:
from gfrl.base.run_my_ppo2 import create_single_scenic_environment

scenario = "../_scenarios/academy/wb/pass_n_shoot_wb.scenic"

num_interactions=10000
expert_file_name = f"../_data/pns_{num_interactions}.npz"

obs, act, rew = generate_expert_data_with_rewards(scenario_file=scenario, num_interactions=num_interactions, file_name=expert_file_name)

  0%|          | 0/10000 [00:00<?, ?it/s]

Environment will ignore actions passed to step() and take action provided by Scenic


100%|██████████| 10000/10000 [03:15<00:00, 51.10it/s]


Expert observation shape:  (10000, 72, 96, 16)
Expert actions shape:  (10000, 19)
Num Episode:  261
Mean Reward:  0.5862068965517241


In [10]:
from gfrl.common.mybase.cloning.dataset import GFDset
dataset = GFDset(expert_file_name)
print(dataset.num_epi, dataset.mean_reward, dataset.size, dataset.acts.shape)

261 0.5862068965517241 10000 (10000, 19)


In [11]:
"""
scenario = "../_scenarios/academy/wb/pass_n_shoot_wb.scenic"

num_interactions=20000
expert_file_name = f"../_data/pns_{num_interactions}.npz"

obs, act, rew = generate_expert_data_with_rewards(scenario_file=scenario, num_interactions=num_interactions, file_name=expert_file_name)
from gfrl.common.mybase.cloning.dataset import GFDset
dataset = GFDset(expert_file_name)
print(dataset.num_epi, dataset.mean_reward, dataset.size, dataset.acts.shape)
"""

'\nscenario = "../_scenarios/academy/wb/pass_n_shoot_wb.scenic"\n\nnum_interactions=20000\nexpert_file_name = f"../_data/pns_{num_interactions}.npz"\n\nobs, act, rew = generate_expert_data_with_rewards(scenario_file=scenario, num_interactions=num_interactions, file_name=expert_file_name)\nfrom gfrl.common.mybase.cloning.dataset import GFDset\ndataset = GFDset(expert_file_name)\nprint(dataset.num_epi, dataset.mean_reward, dataset.size, dataset.acts.shape)\n'

In [12]:
scenario = "../_scenarios/academy/wb/pass_n_shoot_wb.scenic"

num_interactions=50000
expert_file_name = f"../_data/pns_{num_interactions}.npz"

obs, act, rew = generate_expert_data_with_rewards(scenario_file=scenario, num_interactions=num_interactions, file_name=expert_file_name)

from gfrl.common.mybase.cloning.dataset import GFDset
dataset = GFDset(expert_file_name)
print(dataset.num_epi, dataset.mean_reward, dataset.size, dataset.acts.shape)

Environment will ignore actions passed to step() and take action provided by Scenic


100%|██████████| 50000/50000 [41:24<00:00, 20.12it/s] 


Expert observation shape:  (50000, 72, 96, 16)
Expert actions shape:  (50000, 19)
Num Episode:  1266
Mean Reward:  0.5466034755134281
1266 0.5466034755134281 50000 (50000, 19)
